In [ ]:
# 通过add_advanced_features函数添加了一些特征

In [ ]:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm.auto import tqdm
import warnings
import os
import pickle
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')


In [ ]:
is_debug = True
# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction")
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 2 if is_debug else 200
    NFILE = 2 if is_debug else 19
    PATIENCE = 30
    LEARNING_RATE = 1e-3
    
    WINDOW_SIZE = 10  # 与推理保持一致
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94  # 与推理保持一致
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

# ============================================================================
# FEATURE ENGINEERING (与推理完全一致)
# ============================================================================

def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0
    
def add_advanced_features(df):
    """Original nflnn.py features"""
    print("Adding advanced features...")
    df = df.copy()
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # 位置差分
    if 'distance_to_ball' in df.columns:
        df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
        df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
        df['time_to_intercept'] = (df['distance_to_ball'] / 
                                    (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)
    
    # 朝向球的特征
    if 'ball_direction_x' in df.columns:
        df['velocity_alignment'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
        df['velocity_perpendicular'] = (
            df['velocity_x'] * (-df['ball_direction_y']) +
            df['velocity_y'] * df['ball_direction_x']
        )
        if 'acceleration_x' in df.columns:
            df['accel_alignment'] = (
                df['acceleration_x'] * df['ball_direction_x'] +
                df['acceleration_y'] * df['ball_direction_y']
            )
    
    # 速度差分
    if 'velocity_x' in df.columns:
        df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
        df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
        df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
        df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
        df['direction_change'] = df['direction_change'].apply(
            lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
        )
    
    # 距离边界位置
    df['dist_from_left'] = df['y']
    df['dist_from_right'] = 53.3 - df['y']
    df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])
    
    # 球员职位特征
    if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
        df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
        df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
        df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']
    
    # 时间特征
    df['frames_elapsed'] = df.groupby(gcols).cumcount()
    df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
        lambda x: x / (x.max() + 1)
    )
    
    print(f"Total features after enhancement: {len(df.columns)}")
    
    return df


def prepare_sequences_fixed(input_df, output_df=None, test_template=None, is_training=True, window_size=8):
    input_df = input_df.copy()
    
    # Basic features
    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    
    # Velocity
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    delta_t = 0.1
    input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
    input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
    
    # Acceleration
    input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
    input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
    
    # Roles
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    
    # Physics
    mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
    
    # Ball features
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )
    
    # Sort
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # Lag & EMA Features
    for lag in [1, 2, 3]:
        input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag)
        input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag)
        input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag)
        input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag)
    
    input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_x_roll'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    input_df['velocity_y_roll'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    
    print("Step 2/4: Adding advanced features...")
    input_df = add_advanced_features(input_df)

    feature_cols = [
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
        'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight',
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
        'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
        'velocity_x_roll', 'velocity_y_roll',

        'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
        'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
        'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
        'dist_from_sideline', 'dist_from_endzone',
        'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
        'frames_elapsed', 'normalized_time',
    ]
    
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    print(f"使用 {len(feature_cols)} 个特征" if not is_training else f"使用 {len(feature_cols)} 个特征 (训练)")
    
    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    if is_training:
        target_rows = output_df
        target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
        sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
        
        for _, row in tqdm(target_groups.iterrows()):
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                if is_training:
                    continue
                pad_len = window_size - len(input_window)
                pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
                input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            if np.isnan(seq).any():
                if is_training:
                    continue
                seq = np.nan_to_num(seq, nan=0.0)
            
            sequences.append(seq)
            
            out_grp = target_rows[
                (target_rows['game_id']==row['game_id']) &
                (target_rows['play_id']==row['play_id']) &
                (target_rows['nfl_id']==row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
            
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id']
            })
        
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    else:
        target_groups = test_template[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
        sequences, sequence_ids = [], []
        for _, row in target_groups.iterrows():
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                pad_len = window_size - len(input_window)
                pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
                input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            if np.isnan(seq).any():
                seq = np.nan_to_num(seq, nan=0.0)
            
            sequences.append(seq)
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id']
            })
        
        return sequences, sequence_ids


In [ ]:

# ============================================================================
# LOSS (与训练一致)
# ============================================================================

class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        
        huber = torch.where(
            abs_err <= self.delta,
            0.5 * err * err,
            self.delta * (abs_err - 0.5 * self.delta)
        )
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L)
            huber = huber * weight
            mask = mask * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

# ============================================================================
# MODEL (与训练一致)
# ============================================================================

class ImprovedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        out = self.head(ctx)
        out = torch.cumsum(out, dim=1)
        return out


In [ ]:

# ============================================================================
# TRAINING (核心训练逻辑)
# ============================================================================

def prepare_targets(batch_axis, max_h):
    tensors, masks = [], []
    for arr in batch_axis:
        L = len(arr)
        padded = np.pad(arr, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        tensors.append(torch.tensor(padded))
        masks.append(torch.tensor(mask))
    return torch.stack(tensors), torch.stack(masks)

def train_model(X_train, y_train, X_val, y_val, input_dim, horizon, config):
    device = config.DEVICE
    model = ImprovedSeqModel(input_dim, horizon).to(device)
    
    criterion = TemporalHuber(delta=0.5, time_decay=0.03)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, verbose=True)
    
    # Batches
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_train[j] for j in range(i, end)], horizon)
        train_batches.append((bx, by, bm))
    
    val_batches = []
    for i in range(0, len(X_val), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_val))
        bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_val[j] for j in range(i, end)], horizon)
        val_batches.append((bx, by, bm))
    
    best_loss, best_state, bad = float('inf'), None, 0
    
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        train_losses = []
        
        for bx, by, bm in train_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            pred = model(bx)
            loss = criterion(pred, by, bm)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for bx, by, bm in val_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                pred = model(bx)
                loss = criterion(pred, by, bm)
                val_losses.append(loss.item())
        
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{config.EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
            print(f"  ✅ Best val loss improved to {best_loss:.4f}, saving model")
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  ❌ Early stopping at epoch {epoch} (no improvement for {bad} epochs)")
                break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model, best_loss


In [ ]:

# ============================================================================
# MAIN TRAINING FLOW
# ============================================================================


print("="*80)
print("NFL Big Data Bowl 2026 - TRAINING PIPELINE")
print("="*80)

# ===== 1. 加载数据 =====
print("\n[1/5] 加载数据...")
train_input_files = [Config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, Config.NFILE)]  # 只用第1周数据
train_output_files = [Config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, Config.NFILE)]

train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])

test_input = pd.read_csv(Config.DATA_DIR / "test_input.csv")
test_template = pd.read_csv(Config.DATA_DIR / "test.csv")

print(f"训练数据: {len(train_input):,} 行, {len(train_output):,} 行输出")
print(f"测试模板: {len(test_input):,} 行")

# ===== 2. 特征工程 =====
print("\n[2/5] 特征工程...")
sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = prepare_sequences_fixed(
    train_input, train_output, is_training=True, window_size=Config.WINDOW_SIZE
)

sequences = np.array(sequences, dtype=object)
targets_dx = np.array(targets_dx, dtype=object)
targets_dy = np.array(targets_dy, dtype=object)

print(f"生成序列数量: {len(sequences):,}")


In [ ]:

# ===== 3. 训练模型 (5折交叉验证) =====
print("\n[3/5] 开始训练模型...")
groups = np.array([d['game_id'] for d in sequence_ids])
gkf = GroupKFold(n_splits=Config.N_FOLDS)

models_x, models_y, scalers = [], [], []

for fold, (tr_idx, va_idx) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*40}")
    print(f"Fold {fold}/{Config.N_FOLDS}")
    print(f"{'='*40}")
    
    X_tr = sequences[tr_idx]
    X_va = sequences[va_idx]
    y_tr_dx = [targets_dx[i] for i in tr_idx]
    y_va_dx = [targets_dx[i] for i in va_idx]
    y_tr_dy = [targets_dy[i] for i in tr_idx]
    y_va_dy = [targets_dy[i] for i in va_idx]
    
    # 特征标准化 (只用训练集fit)
    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_tr]))
    
    # 标准化数据
    X_tr_scaled = np.stack([scaler.transform(s) for s in X_tr])
    X_va_scaled = np.stack([scaler.transform(s) for s in X_va])
    
    input_dim = X_tr[0].shape[1]  # 特征维度
    
    # ===== 训练 X 方向模型 =====
    print("\n🔵 训练 X 方向模型...")
    model_x, best_loss_x = train_model(
        X_tr_scaled, y_tr_dx, X_va_scaled, y_va_dx,
        input_dim, Config.MAX_FUTURE_HORIZON, Config
    )
    models_x.append(model_x)
    scalers.append(scaler)
    print(f"✅ X 方向最佳 Val Loss: {best_loss_x:.4f}")
    
    # ===== 训练 Y 方向模型 =====
    print("\n🔴 训练 Y 方向模型...")
    model_y, best_loss_y = train_model(
        X_tr_scaled, y_tr_dy, X_va_scaled, y_va_dy,
        input_dim, Config.MAX_FUTURE_HORIZON, Config
    )
    models_y.append(model_y)
    print(f"✅ Y 方向最佳 Val Loss: {best_loss_y:.4f}")

# ===== 4. 保存模型 =====
print("\n[4/5] 保存模型...")
MODEL_SAVE_DIR = Path("./models/")
MODEL_SAVE_DIR.mkdir(exist_ok=True)

for fold in range(Config.N_FOLDS):
    # 保存模型X
    torch.save(models_x[fold].state_dict(), MODEL_SAVE_DIR / f"model_x_fold{fold+1}.pth")
    # 保存模型Y
    torch.save(models_y[fold].state_dict(), MODEL_SAVE_DIR / f"model_y_fold{fold+1}.pth")
    # 保存Scaler
    joblib.dump(scalers[fold], MODEL_SAVE_DIR / f"scaler_fold{fold+1}.pkl")
    
    print(f"已保存 Fold {fold+1} 模型到 {MODEL_SAVE_DIR}")

print(f"\n🎉 训练完成！模型已保存到 {MODEL_SAVE_DIR}")
print(f"📂 请将此文件夹压缩并上传为 Kaggle Dataset，命名为如: nfl-models")
print(f"🔗 推理时请将模型上传到 /kaggle/input/nfl-models/ 目录")
print(f"📝 推理notebook会自动从该目录加载模型进行预测")



In [ ]:
# !rm -rf /kaggle/working/models/
# !rm /kaggle/working/nfl_models.zip

# 打包模型权重
# 下载到本地再导入推理Notebook
!zip -r nfl_models.zip /kaggle/working/models